In [1]:
from sklearn.datasets import load_diabetes
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import torch
import nltk
import torch.nn as nn 
import json
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import os

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fjordrunner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
with open('songs_training_data.json', 'r') as fp:
    songs_training_data = json.load(fp)

In [3]:
print(len(songs_training_data))

23014


In [4]:
word_counts = Counter()

for k, v in songs_training_data.items():
    lyrics = v['lyrics']
    tokens = nltk.word_tokenize(lyrics)
    word_counts = word_counts + Counter(tokens)

shortened_word_count_info = word_counts.most_common(5000)

In [5]:
print(shortened_word_count_info)

[('i', 245771), ('you', 205742), ('the', 202968), ('and', 119580), ('to', 116356), ('a', 104372), ('it', 95566), ('me', 83530), ('not', 70762), ('my', 69588), ('in', 69520), ('is', 59591), ('of', 58687), ('do', 55415), ('that', 55240), ('your', 50934), ('on', 48713), ('we', 44099), ('all', 40304), ('are', 40035), ('be', 37739), ('love', 37161), ('for', 37143), ('am', 36583), ('will', 32694), ('know', 31463), ('so', 31189), ('have', 30820), ('but', 30758), ('no', 29252), ('like', 27859), ('with', 27701), ('just', 27687), ('what', 25892), ('this', 25833), ('got', 25730), ('oh', 24642), ('when', 24188), ('up', 24175), ('na', 23105), ('can', 21778), ('now', 21135), ('get', 20951), ('go', 20845), ('she', 20185), ('there', 19713), ('was', 19319), ('out', 19266), ('time', 19136), ('if', 19052), ('they', 19052), ('yeah', 18829), ('come', 18829), ('down', 18782), ('one', 18732), ("n't", 17481), ('he', 16672), ('see', 16454), ('want', 16174), ('let', 15738), ('never', 15683), ("'s", 15131), ('sa

In [6]:
# TODO: add [UNK] token
 
word_to_idx = {tup[0]: i for i, tup in enumerate(shortened_word_count_info)}
word_to_idx["<UNK>"] = len(shortened_word_count_info)

idx_to_word =  dict((v,k) for k,v in word_to_idx.items())
idx_to_word[len(shortened_word_count_info)] = "<UNK>"

In [7]:
print(len(word_to_idx))

5001


In [8]:
print(word_to_idx)

{'i': 0, 'you': 1, 'the': 2, 'and': 3, 'to': 4, 'a': 5, 'it': 6, 'me': 7, 'not': 8, 'my': 9, 'in': 10, 'is': 11, 'of': 12, 'do': 13, 'that': 14, 'your': 15, 'on': 16, 'we': 17, 'all': 18, 'are': 19, 'be': 20, 'love': 21, 'for': 22, 'am': 23, 'will': 24, 'know': 25, 'so': 26, 'have': 27, 'but': 28, 'no': 29, 'like': 30, 'with': 31, 'just': 32, 'what': 33, 'this': 34, 'got': 35, 'oh': 36, 'when': 37, 'up': 38, 'na': 39, 'can': 40, 'now': 41, 'get': 42, 'go': 43, 'she': 44, 'there': 45, 'was': 46, 'out': 47, 'time': 48, 'if': 49, 'they': 50, 'yeah': 51, 'come': 52, 'down': 53, 'one': 54, "n't": 55, 'he': 56, 'see': 57, 'want': 58, 'let': 59, 'never': 60, "'s": 61, 'say': 62, 'way': 63, 'from': 64, 'make': 65, 'feel': 66, 'ca': 67, 'would': 68, 'take': 69, 'at': 70, 'back': 71, 'as': 72, "'m": 73, 'babi': 74, 'her': 75, 'gon': 76, "'": 77, 'day': 78, 'need': 79, 'could': 80, 'away': 81, 'here': 82, 'how': 83, 'heart': 84, 'been': 85, 'life': 86, 'right': 87, 'night': 88, 'look': 89, 'by': 

In [9]:
max_length = 0
for k, v in songs_training_data.items():
    lyrics = v['lyrics']
    tokens = nltk.word_tokenize(lyrics)
    # TODO: include the [UNK] token for words not seen in the corpus, (make and if else statement)
    numerical_form = []
    for word in tokens:
      if word in word_to_idx:
        numerical_form.append(word_to_idx[word])
      else:
        numerical_form.append(word_to_idx["<UNK>"])

        
    # numerical_form = [word_to_idx[word] for word in tokens if word in word_to_idx ]
    # print(len(numerical_form))

    songs_training_data[k]['tensor'] = torch.tensor(numerical_form)
    max_length = max(max_length, songs_training_data[k]['tensor'].size()[0])

In [10]:
print(len(shortened_word_count_info))

5000


In [11]:
print(len(songs_training_data))

23014


In [12]:
lyrics = [songs_training_data[k]['tensor'] for k in songs_training_data]
print(len(lyrics))

lyrics = pad_sequence(lyrics, batch_first= True, padding_value=len(shortened_word_count_info))

tags = np.array([songs_training_data[k]['tag_list'] for k in songs_training_data])

23014


In [13]:
tags_updated = tags.argmax(1)
tags_updated = np.where(tags_updated > 12, tags_updated - 1, tags_updated)
tags_updated.shape

(23014,)

In [14]:
class trainingdataset(Dataset):
    def __init__(self, lyrics, tags):
        self.x = torch.tensor(lyrics)
        self.y = torch.tensor(tags)
        self.length = self.x.shape[0]

    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return self.length


training_dataset = trainingdataset(lyrics, tags_updated)

/Users/fjordrunner/cs4300-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
class lyric_based_training_dataset(Dataset):
    def __init__(self, lyrics):
        self.x = torch.tensor(lyrics)
        self.length = self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index]
    
    def __len__(self):
        return self.length

training_dataset = lyric_based_training_dataset(lyrics)

/Users/fjordrunner/cs4300-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
len(training_dataset), training_dataset[0]

(23014, tensor([ 770,    0,  266,  ..., 5000, 5000, 5000]))

In [17]:
dataloader = DataLoader(dataset=training_dataset, shuffle=True, batch_size=1)

In [18]:
from itertools import chain
from sklearn.utils import class_weight

train_labels = list(tags_updated)
label_ids = np.unique(train_labels)
print(len(label_ids))

class_weights = class_weight.compute_class_weight('balanced', classes=label_ids, y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights, len(class_weights)

13


(tensor([5.1763, 0.8154, 5.9208, 0.6948, 0.9342, 0.7608, 1.4129, 1.4028, 0.5461,
         1.1921, 1.8028, 1.0755, 0.4970]), 13)

In [19]:
print(label_ids)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12]


In [20]:
print(len(shortened_word_count_info))

5000


In [24]:
class net(nn.Module):
  def __init__(self, embedding_dim = 500, hidden_dim=250, hidden2_dim = 100, padding_idx=len(shortened_word_count_info),
               output_size=len(class_weights)):
    super(net,self).__init__()
    self.l1 = nn.Embedding(num_embeddings=len(shortened_word_count_info) + 1, 
                           embedding_dim=embedding_dim, padding_idx=padding_idx)
    nn.init.xavier_uniform_(self.l1.weight.data)
    self.l1.weight.data[padding_idx] = torch.zeros(embedding_dim)

    self.l2 = nn.Linear(embedding_dim, hidden_dim)
    nn.init.xavier_uniform_(self.l2.weight.data)

    self.relu = nn.GELU()
    self.l3 = nn.Linear(hidden_dim, hidden2_dim)

    self.l4 = nn.Linear(hidden2_dim, output_size)

  def forward(self, x):
    # x: (b, L)
    # o: (b, L, e)
    output1 = self.l1(x) 
    # o: (b, e)
    # print(output.shape)
    # 500-dimensional
    output2 = torch.mean(output1, 1)
    # o: (b, h)
    #250-dimensional
    output3 = self.relu(self.l2(output2))
    # o: (b, o)

    output4 = self.relu(self.l3(output3))

    output = self.l4(output4)
    # (b, 9)

    return output, output3, output4

In [25]:
model = torch.load('model.pt', map_location=torch.device('cpu'))

In [26]:
mat_100 = []


for x_train in dataloader:
  _, __out, _100_output =  model(x_train.cpu())
  mat_100.append(_100_output)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
for i, tens in enumerate(mat_100):
    mat_100[i] = tens.tolist()

In [65]:
# for i, tens in enumerate(mat_250):
#     mat_250[i] = tens.tolist()

In [ ]:
with open('./100_lyric_embeddngs.json', 'w') as fp:
    json.dump(mat_100, fp)

In [66]:
# with open('./250_lyric_embeddngs.json', 'w') as fp:
#     json.dump(mat_250, fp)

In [58]:
print(len(mat_500))

32028


In [67]:
print(len(mat_250))

32028
